# TP 3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [32]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


### Récupération des données

In [33]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Programmation dynamique

In [96]:
function constructionC(price, weight, capacity)
    # Fonction pour construire la matrice C

    # Initialisation de la matrice
    c = zeros(length(price),capacity+1)

    i = 1

    while(i<= length(price))
        # Initialisation des colonnes à 2 car la première colonne reste à 0
        j = 2

        while(j<=capacity+1)
            # Définition des lignes et colonnes précédentes ou en enlevant le poids
            # pour éviter de sortir de la matrice
            i_prec = max(i-1,1)
            j_moins_poids = max(1,j-weight[i])

            # Calcul C récurrence pour comparaison
                # Cas sans ajout du poids
            c_prec = c[i_prec,j]
                # Cas ajout du poids à partir du cas où le poids max était plus faible
            c_actuel_ajout = c[i_prec,j_moins_poids]+price[i]
                    # Test admissibilité, respect contrainte
            if (weight[i]>j-1)
                c_actuel_ajout = 0
            end
            # Comparaison des C obtenues (relation récurrence) pour maximiser le gain
            c[i,j] = max(c_prec,c_actuel_ajout)

            # Avancement colonne
            j+=1

        end
        # Avancement ligne
        i+=1

    end

    return c

end

function analyseC(c,price,weight,capacity)
    # Initialisation des résultats
    Bestsol, ObjSol, BestProfit = zeros(length(price)), Any[], -1
    # Début analyse dans le cas de poids correspondant à l'étude et de gain maximal
    i, j = length(price), capacity + 1
    # Récupération meilleur profit
    BestProfit = c[i,j]
    # Parcours descendant des lignes (i décroit)
    while (i > 1)
        # Si le gain est égal à celui de la ligne précédente, alors l'objet n'a pas été pris
        if c[i-1,j] == c[i,j]
            Bestsol[i] = 0.0
        # Sinon le gain à augmenter de la valeur ajoutée de l'objet
        else
            Bestsol[i] = 1.0
            push!(ObjSol,i)
            # On observe le gain pour un sac de capacité moindre et donc de poids occupé maximal sans l'objet validé
            j -= weight[i]
            # Vérification de non débordement de la matrice
            if j <= 0
                j = 1
            end
        end
        # Analyse ligne suivante : si l'objet précédent est présent ou non
        i-=1
    end

    # Analyse du premier objet potentiellement ajouté à part car pas possible d'observer les lignes précédentes.
    if c[i,j] != 0
        # Si le gain est non nul, alors ce premier objet est ajouté au sac
        Bestsol[i] = 1.0
    end 

    return Bestsol, ObjSol, BestProfit
end

function SolveKnapInstanceDym(filename, affichage)

    price, weight, capacity = readKnaptxtInstance(filename)
    if affichage
        println("\nConstruction de la matrice C ...")
    end
    c = constructionC(price, weight, capacity)
    if affichage
        println("\n... Matrice C contruite")
    end
    #println("\nMatrice C obtenue ",c)
    if affichage
        println("\nAnalyse de la matrice C ...\n")
    end
    Bestsol,ObjSol,BestProfit = analyseC(c,price, weight, capacity)
    if affichage
        println("\n... Matrice C analysée")

        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal (Matrice 0 objet pas présent, 1 objet présent) sol=", Bestsol, "\n\nOptima (objet présent) x=",ObjSol)
    end
    return BestProfit,Bestsol,ObjSol

end

SolveKnapInstanceDym (generic function with 1 method)

### Affichage du résultat final

In [97]:
cd("/home/romai/Enseeiht/S7/RechOp/TPs/instancesETU_TP3/")

In [98]:

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/test.opb.txt",true)

println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal (Matrice 0 objet pas présent, 1 objet présent) sol=", Bestsol, "\n\nOptima (objet présent) x=",ObjSol)



Construction de la matrice C ...

... Matrice C contruite

Analyse de la matrice C ...


... Matrice C analysée

******

Optimal value = 65.0

Optimal (Matrice 0 objet pas présent, 1 objet présent) sol=[0.0, 1.0, 0.0, 1.0]

Optima (objet présent) x=Any[4, 2]
  0.084354 seconds (24.43 k allocations: 1.188 MiB, 99.17% compilation time)

******

Optimal value = 65.0

Optimal (Matrice 0 objet pas présent, 1 objet présent) sol=[0.0, 1.0, 0.0, 1.0]

Optima (objet présent) x=Any[4, 2]


Par construction récurcive de la matrice C, pour chaque valeur $C_{i,j}$, on a la valeur maximale par ajout ou non des objets $k \in [1;...;i]$ respectant la capacité maximale $j$. Ainsi la valeur de la dernière ligne et de la dernière colonne correspond à la valeur maximale obtenue après ajout ou non de tout les poids dans le cas de la capacité maximale de l'étude de départ. En inversant la construction de la matrice, on peut retrouver les objets ajoutés dans le sac et ainsi avoir la meilleur solution.

### Programme Branch-and-Bound (TP2)

In [135]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end
function TestsSondabilite(listobjs, listvals, price, weight, capacity, borneinf, bornesup, BestProfit, Bestsol, affichage)
    TA, TO, TR = false, false, false
    
    current_weight = 0
    current_price = 0
    for i in 1:length(listobjs)
        if listvals[i] == 1
            current_weight += weight[listobjs[i]]
            current_price += price[listobjs[i]]
        end
               
    end

    if current_weight > capacity  # Test d'Admissibilité
        TA = true
        if affichage
            println("TA")
        end
    elseif (bornesup < BestProfit)  # Test d'optimalite
        TO = true
        if affichage
            println("TO")
        end
    elseif borneinf == bornesup # Test de resolution
        TR = true
        if affichage
            println("TR")
        end
        if bornesup >= BestProfit
            Bestsol = copy(listobjs.*listvals)
            BestProfit = bornesup
            if affichage
                println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
            end
        end
    else
        if affichage
            println("non sondable")
        end
    end
    return TA, TO, TR, Bestsol, BestProfit
end

function SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, n,  price, weight, affichage)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left  
    
    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj, maxr = 1, 0, -1.0
    while((i <= n))
        if (!(i in listobjs)) && ((price[i]/weight[i]) >= maxr)
            maxr = (price[i]/weight[i])
            obj = i
        end        
        i+=1
    end

    if affichage
        println("\nbranch on object ", obj, "\n")
    end
    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes, affichage)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            if affichage
                println("\nFINISHED")
            end
            stop=true
        end
    else
        #the root node was sondable
        if affichage
            println("\nFINISHED")
        end
        stop=true
    end
    return stop 
end
function Bornes_1!(price, weight, capacity, listobjs, listvals)
    
    placeDispo, prixEnCours, obj, maxr = capacity, 0.0, 0, -1.0
    for i in 1:length(listobjs)
        if listvals[i] == 1
            placeDispo -= weight[listobjs[i]]
            prixEnCours += price[listobjs[i]]
        end
    end 

    if (length(listobjs) < length(price))
        for i in 1:length(price)
            if (!(i in listobjs)) && ((price[i]/weight[i]) >= maxr)
                maxr = (price[i]/weight[i])
                obj = i
            end
        end
        facteurObj = placeDispo/weight[obj]

        borneinf, bornesup = prixEnCours, (prixEnCours+facteurObj*price[obj])
    else 
        borneinf, bornesup = prixEnCours, prixEnCours
    end
    return borneinf, bornesup

end

function Bornes_2!(price, weight, capacity, listobjs, listvals)
    placeDispo, prixEnCours, listeAjoutee, obj = capacity, 0.0, [], -1
    for i in 1:length(listobjs)
        if listvals[i] == 1
            placeDispo -= weight[listobjs[i]]
            prixEnCours += price[listobjs[i]]
        end
    end
    
    if (length(listobjs) < length(price)) && (placeDispo > 0)
        prixMax = prixEnCours
        while (placeDispo > 0) && ((length(listobjs) + length(listeAjoutee)) < length(price)) 
            maxr, obj = -1.0, 0
            for i in 1:length(price)
                if (!(i in listobjs)) && (!(i in listeAjoutee)) && ((price[i]/weight[i]) >= maxr)
                    maxr = (price[i]/weight[i])
                    obj = i
                end
            end
            push!(listeAjoutee,obj)

            placeDispo -= weight[obj]
            prixMax += price[obj]
        end
        
        if placeDispo < 0    
            placeDispo += weight[obj]
            prixMax -= price[obj]
        end
        
        facteurObj = placeDispo/weight[obj]  
        return prixEnCours,prixMax+price[obj]*facteurObj
    else
        return prixEnCours, prixEnCours
    end
end
function SolveKnapInstanceBranch(filename, borne, affichage)

    price, weight, capacity = readKnaptxtInstance(filename)

    #model2, x = CreationModele_LP(price, weight, capacity)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]
    Bestvals=[]

    current_node_number=0
    stop = false

    while(!stop)
        if affichage
            println("\nNode number ", current_node_number, ": \n---------------\n")
        end
        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        #=
        #create LP of current node
        MajModele_LP!(model2, x, listobjs, listvals)
        
        println(model2)
        
        print("Solve the LP model of the current node to compute its bound: start ... ")

        status = optimize!(model2)

        println("... end"); 

        print(": Solution LP")
        if(termination_status(model2) == MOI.INFEASIBLE)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print(" ", objective_value(model2))
            [print("\t", name(v),"=",value(v)) for v in all_variables(model2)] 
        end
        println(" "); 

        =#

        #println("listobjs, listvals, listnodes", listobjs, listvals, listnodes)
        #Calcul des bornes
        if borne == "borne1"
            borneinf, bornesup = Bornes_1!(price, weight, capacity, listobjs, listvals)
        else borne == "borne2"
            borneinf, bornesup = Bornes_2!(price, weight, capacity, listobjs, listvals)
        end
        if (affichage==true)
            println("bornes :", [borneinf, bornesup])
            println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
        
        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite(listobjs, listvals, price, weight, capacity, borneinf, bornesup, BestProfit, Bestsol, affichage)
        
        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        
        if(!is_node_sondable)
            SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, length(price), price, weight, affichage)
        else
            stop = ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes, affichage)        
        end
        
        
        
        #Reset_allLP!(model2, x)

        current_node_number = current_node_number + 1
    end
    if affichage
        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
    end
    return BestProfit, filter(p->(>0),Bestsol), trParentnodes, trChildnodes, trNamenodes, listvals

end

SolveKnapInstanceBranch (generic function with 1 method)

In [136]:
println("\n******\n Cas dynamique\n")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound\n")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2, listvals = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)


******
 Cas dynamique



  0.012430 seconds (264.19 k allocations: 4.428 MiB)

Optimal value = 1894.0
Optimal x=Any[47, 44, 30, 27, 25, 21, 20, 13, 2]

******
 Cas Branch-and-Bound



  0.227321 seconds (359.24 k allocations: 13.434 MiB, 83.37% compilation time)

Optimal value = 1894.0
Optimal x=[30.0, 21.0, 13.0, 47.0, 27.0, 2.0, 25.0, 44.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 20.0]


In [137]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)


******
 Cas dynamique
  0.012223 seconds (264.19 k allocations: 4.428 MiB)

Optimal value = 1894.0
Optimal x=Any[47, 44, 30, 27, 25, 21, 20, 13, 2]

******
 Cas Branch-and-Bound


  0.036783 seconds (158.64 k allocations: 2.828 MiB)

Optimal value = 1894.0
Optimal x=[30.0, 21.0, 13.0, 47.0, 27.0, 2.0, 25.0, 44.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 20.0]


In [138]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/similar_weights/knapPI_9_100_1000_2_-997.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/similar_weights/knapPI_9_100_1000_2_-997.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)


******
 Cas dynamique


 10.834877 seconds (296.47 M allocations: 4.713 GiB, 5.43% gc time)

Optimal value = 997.0
Optimal x=Any[119]

******
 Cas Branch-and-Bound


  3.403447 seconds (2.59 M allocations: 42.404 MiB)

Optimal value = 997.0
Optimal x=[119.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [139]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/weakly_correlated/knapPI_2_100_1000_1_-1514.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/weakly_correlated/knapPI_2_100_1000_1_-1514.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)


******
 Cas dynamique
  0.024126 seconds (532.09 k allocations: 8.897 MiB)

Optimal value = 1514.0
Optimal x=Any[85, 71, 57, 49, 45, 38, 33, 24, 11]

******
 Cas Branch-and-Bound


  0.159441 seconds (351.73 k allocations: 5.881 MiB)

Optimal value = 1514.0
Optimal x=[38.0, 24.0, 0.0, 33.0, 71.0, 57.0, 85.0, 45.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 49.0]


In [140]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/multiple_strongly_correlated/knapPI_14_50_1000_1_-2586.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/multiple_strongly_correlated/knapPI_14_50_1000_1_-2586.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)


******
 Cas dynamique
  0.012756 seconds (262.90 k allocations: 4.409 MiB)

Optimal value = 3392.0
Optimal x=Any[45, 43, 15]

******
 Cas Branch-and-Bound
  0.003157 seconds (14.97 k allocations: 299.321 KiB)

Optimal value = 2586.0
Optimal x=[15.0, 45.0, 43.0, 1.0, 32.0, 22.0, 30.0, 0.0, 46.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [141]:

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt",true)


Construction de la matrice C ...

... Matrice C contruite

Analyse de la matrice C ...


... Matrice C analysée

******

Optimal value = 1894.0

Optimal (Matrice 0 objet pas présent, 1 objet présent) sol=[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]

Optima (objet présent) x=Any[47, 44, 30, 27, 25, 21, 20, 13, 2]
  0.011411 seconds (266.39 k allocations: 4.495 MiB)


(1894.0, [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], Any[47, 44, 30, 27, 25, 21, 20, 13, 2])

In [142]:
BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/uncorrelated/knapPI_1_50_1000_1_-8373.opb.txt",true)


Construction de la matrice C ...

... Matrice C contruite

Analyse de la matrice C ...


... Matrice C analysée

******

Optimal value = 8373.0

Optimal (Matrice 0 objet pas présent, 1 objet présent) sol=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

Optima (objet présent) x=Any[49, 39, 38, 33, 31, 26, 24, 14, 13, 11, 7]


(8373.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], Any[49, 39, 38, 33, 31, 26, 24, 14, 13, 11, 7])

In [143]:
BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/weakly_correlated/knapPI_2_50_1000_1_-1396.opb.txt",true)


Construction de la matrice C ...

... Matrice C contruite

Analyse de la matrice C ...


... Matrice C analysée

******

Optimal value = 1396.0

Optimal (Matrice 0 objet pas présent, 1 objet présent) sol=[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Optima (objet présent) x=Any[45, 38, 37, 33, 24, 4]


(1396.0, [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], Any[45, 38, 37, 33, 24, 4])

In [145]:
#=cd("instancesETU/KNAPnewformat")
listeDossier = readdir()
for dossier in listeDossier
    if isdir(dossier)
        cd(dossier)
        listeTest = readdir()
        for test in listeTest
            println("\n",test,"\n")
            _, _ = SolveKnapInstanceDym(test,false)
            #=try 
                
            catch e
                println("Error test")
            end=#
        end
        cd("..")
    end
end=#